# Imports

In [13]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision import transforms
from tqdm import tqdm_notebook as tqdm
import torch.optim as optim
from torchvision.utils import make_grid
from helpers import upconv, conv, get_noise, train, build_dataloader
from loss import *

device = 'cuda'

# Models

In [18]:
class Generator(nn.Module):
    def __init__(self,z_dim, im_chan=3, activation_func=nn.ReLU()):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        self.dropout = nn.Dropout(0.4)
        self.activation_func = activation_func
        self.upconv1 = upconv(64, 32, up=True, mode='nearest')
        self.upconv2 = upconv(32, 16,  up=True)
        self.upconv3 = upconv(16, 4,   up=True, scale_factor=4)
        self.upconv4 = upconv(4, 3,  up=True, scale_factor=4)
        self.upconv5 = upconv(3, 3,  up=True)
        self.upconv6 = upconv(3, 3,  batch_norm=False)
    
    def forward(self, noise):
        x = noise.view(len(noise), self.z_dim, 1, 1)
        
        x = self.upconv1(x)
        x = self.activation_func(x)
    
        x = self.upconv2(x)
        x = self.activation_func(x)
        
        x = self.upconv3(x)
        x = x + self.activation_func(x)
        
        x = self.upconv4(x)
        x = x + self.activation_func(x)
        
        x = self.upconv5(x)
        x = self.activation_func(x)
        
        x = self.upconv6(x)
        return x

In [4]:
class Critic(nn.Module):
    
    def __init__(self, im_chan=3, activation_func=nn.ReLU()):
        super(Critic, self).__init__()
        self.activation_func = activation_func
        self.conv1 = conv(im_chan, 16, kernel_size=1, padding=0,batch_norm=False)
        self.conv2 = conv(16, 16, kernel_size=1,  stride=1, padding=0)
        
        self.conv3 = conv(16, 32, kernel_size=4)
        self.conv4 = conv(32, 32, kernel_size=1, stride=1, padding=0)
        
        self.conv5 = conv(32, 64, kernel_size=4)
        self.conv6 = conv(64, 64, kernel_size=1, stride=1, padding=0)
        
        
        self.conv7 = conv(64, 128, kernel_size=3, stride=2, padding=2)
        self.conv8 = conv(128, 128, kernel_size=1, stride=1, padding=0)
        
        
        self.conv9 = conv(128, 128, kernel_size=1, stride=3)
        
        self.conv10 =conv(128, 1, kernel_size=4, stride=3, batch_norm=False)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.activation_func(x)
        x = x + self.conv2(x)
        
        x = self.conv3(x)
        x = self.activation_func(x)
        x = x + self.conv4(x)
        
        x = self.conv5(x)
        x = self.activation_func(x)
        x = x + self.conv6(x)
        
        x = self.conv7(x)
        x = self.activation_func(x)
        
        x = self.conv8(x)
        x = self.activation_func(x)
        
        x = self.conv9(x)
        x = self.activation_func(x)
        
        x = self.conv10(x)
        return x.view(len(x), -1)

# Hyperparameters

In [9]:
FILES_PATH = "/home/workspace/data/img_align_celeba"
LEARNING_RATE = 0.0002
BATCH = 256
Z_DIM= 64
BETA_1 = 0.5
BETA_2 = 0.999
C_LAMBDA = 10

# Build Network

In [10]:
def build_network(gpu=True):
    """
    Builds network    
    """
    # try to change activation between two generator and critic
    G = Generator(z_dim=Z_DIM, activation_func=nn.ReLU())
    C = Critic(activation_func=nn.ReLU())
    if gpu:
        G = Generator(z_dim=Z_DIM, activation_func=nn.ReLU()).cuda()
        C = Critic(activation_func=nn.ReLU()).cuda()

    def weight_init(m):
        if isinstance(m, nn.Conv2d):
            nn.init.normal_(m.weight, 0.0, 0.02)
        if isinstance(m, nn.BatchNorm2d):
            nn.init.normal_(m.weight, 0.0, 0.02)
            nn.init.constant_(m.bias, 0)
    G.apply(weight_init)
    C.apply(weight_init)
    return G, C

# Initialize Network and Optimizers

In [14]:
G, C = build_network()
G_opt = optim.Adam(G.parameters(), lr=LEARNING_RATE, betas=(BETA_1, BETA_2))
C_opt = optim.Adam(C.parameters(), lr=LEARNING_RATE, betas=(BETA_1, BETA_2))

data_loader = build_dataloader(FILES_PATH, batch=BATCH)

# Train

In [23]:
train(data_loader,
      G, C,
      G_opt, C_opt,
      n_epochs = 1,
      device = 'cuda',
      critic_repeat = 1,
      gen_repeat = 1,
      display_step = 500,
      z_dim = Z_DIM,
      c_lambda = C_LAMBDA
     )

# Test <p hidden>😎</p>

In [16]:
G.load_state_dict(torch.load('G.pth'))

In [ ]:
def generate_images(gen_model, no_of_images, num_images=5):
    gen_model = gen_model.eval()
    with torch.no_grad():
        show_tensor_images(gen_model(get_noise(no_of_images, 64)), num_images=num_images)

In [17]:
generate_images(G, no_of_images=1, num_images=1)